## ViCTORIA Results

In [ ]:
import sys
sys.path.insert(1, "model/")
sys.path.insert(1, "lib/")
from ViCTORIA_network import ViCTORIA_Network, coeff_determination
from dataset_utils import encode_position, read_many_hdf5
from ScoreGetter import ScoreGetter

import pickle
import chess
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

import platform
if platform.system() == "Darwin":
    %config InlineBackend.figure_format="retina"  # For high DPI display

We retrieve the model's optimal hyperparameters from the `choose_hyperparameters` notebook.

In [ ]:
input_file = open("model/hyperparameters.pickle", "rb")
hyperparams = pickle.load(input_file)
input_file.close()

We create a model with these hyperparameters and we load the weights learned within the `train_model` notebook.

In [ ]:
model = ViCTORIA_Network(filters=hyperparams["filters"], nb_blocks=hyperparams["nb_blocks"])
model.build((1, 8, 8, 15))
model.compile(loss='mean_absolute_error', optimizer="adam", metrics=[coeff_determination])
_ = model.load_weights("model/weights/weights")

In [ ]:
def plot_regression_results(model, X_test, y_test, path=None):
  preds = model.predict(X_test)
  r2_score = coeff_determination(tf.convert_to_tensor(y_test), preds).numpy()
  ma = np.max(y_test)
  mi = np.min(y_test)
  x = np.linspace(ma, mi, 100)
  plt.figure(figsize=(10,5))
  plt.plot(x, x, "r-.", label = "Predictions = True values")
  plt.scatter(y_test, preds, label = f"$R^2 = {r2_score:.2f}$")
  plt.xlabel("True values (centipawn)")
  plt.ylabel("Predictions (centipawn)")
  plt.title(f"Results of ViCTORIA model with {model.nb_blocks} SE-ResNet blocks and {X_test.shape[0]} test positions")
  plt.legend()
  if path:
    plt.savefig(path)

We load our test dataset and we plot the results (finally).

In [ ]:
directory = "E:/IA/Deep_ViCTORIA/Datasets/SE_ResNet/"
positions_test, scores_test = read_many_hdf5(148970, directory, "_test")
plot_regression_results(model, positions_test, scores_test, path="results/result.pdf")

In [ ]:
def test_position (model, engine_path, fen):
  score_getter = ScoreGetter(engine_path, "eval", "go depth 1")

  print(fen)
  e = encode_position(fen).reshape((1, 8, 8, 15))
  e = e.astype("float32")
  print(f"Stockfish 14 score: {score_getter.getScore(fen)}")
  print(f"ViCTORIA score: {int(model.predict(e).flatten()[0])}")

  return chess.Board(fen)

We observe the difference between ViCTORIA and the Stockfish evaluation function on any chess position.

In [ ]:
engine_path = "bin/stockfish.exe"
test_position(model, engine_path, "rnbqkbnr/pppppppp/8/8/6P1/8/PPPPPP1P/RNBQKBNR b KQkq - 0 1")